Create Agents to Research and Write an Article

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import from the crewAI libray.
from crewai import Agent, Task, Crew

In [6]:
# import library
from dotenv import load_dotenv
import os #needed to call function
import openai

In [7]:
# setup configuration function
def configure():
    load_dotenv()
    
# load configuration
configure()

# get the key
openai_api_key = os.getenv("OPENAI_API_KEY")

In [22]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

Creating Agents
- Define your Agents, and provide them a role, goal and backstory.
- It has been seen that LLMs perform better when they are role playing.

Agent: Planner
Note: The benefit of using multiple strings : 

varname = "line 1 of text"
          "line 2 of text"
versus the triple quote docstring:
varname = """line 1 of text
             line 2 of text
          """
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [9]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

Agent: Writer

In [10]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

Agent: Editor

In [11]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

Creating Tasks
Define your Tasks, and provide them a description, expected_output and agent.

In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

Task: Write

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

Task: Edit

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Creating the Crew
Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    # the order of the tasks is important, 
    # as the output of one task is the input of the next.
    tasks=[plan, write, edit],
    verbose=2
)

Running the Crew

In [23]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention the rapid advancements and innovations in AI technology
- Introduction to the latest trends and key players in the AI industry

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning and Deep Learning advancements
- AI-powered automation in various sectors
- Ethical AI and responsible use of AI technology

In [24]:
# Display the results of your execution as markdown in the notebook.
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

**Introduction:**
Artificial Intelligence (AI) has revolutionized various industries with its cutting-edge technology and innovative applications. The rapid advancements in AI have paved the way for unprecedented growth and efficiency across sectors. In this blog post, we will delve into the latest trends and key players shaping the future of AI.

**Latest Trends in Artificial Intelligence:**
Machine Learning and Deep Learning have been at the forefront of AI advancements, enabling systems to learn and improve from data without explicit programming. AI-powered automation has streamlined processes in sectors such as healthcare, finance, and transportation, increasing productivity and enhancing decision-making. Ethical AI practices are gaining importance, emphasizing the responsible use of AI technology to mitigate potential risks and biases. The integration of AI in various industries is driving innovation and creating new opportunities for growth and development.

**Key Players in the AI Industry:**
Major tech giants like Google, Microsoft, IBM, and Amazon are leading the way in AI research and development. Google's AI initiatives have pushed the boundaries of machine learning and natural language processing. Microsoft's Azure AI platform offers a wide range of AI services for businesses, empowering them to leverage AI capabilities effectively. IBM Watson provides enterprise AI solutions, enabling organizations to harness the power of AI for complex data analysis and decision-making. Amazon Web Services (AWS) offers a suite of AI services, including machine learning, natural language processing, and computer vision. Startups like OpenAI and DeepMind are also making significant contributions to AI technology, driving innovation and pushing the AI industry forward.

**Target Audience Analysis:**
Professionals in the tech industry who are interested in the latest AI advancements will find this blog post insightful. Business owners looking to implement AI solutions in their organizations can gain valuable information about the top players in the AI industry. Students and researchers studying AI technology will benefit from the discussions on AI trends and innovations. Anyone curious about the future implications of AI will find this content engaging and informative.

**SEO Keywords:**
- Artificial Intelligence trends
- Top AI companies
- Future of AI technology
- AI industry updates
- AI innovations 2021

**Call to Action:**
Stay updated on AI news and innovations by following reputable sources and industry reports. Explore AI solutions for your business or personal use to leverage the power of AI technology. For further reading, check out the resources provided in this blog post to deepen your understanding of AI trends and key players in the industry.

In conclusion, the future of Artificial Intelligence is bright, with continuous advancements and innovations shaping the industry. By staying informed and exploring AI solutions, individuals and organizations can harness the full potential of AI technology for growth and success.

(Source: Forbes, Gartner, IEEE)